## Aplicación del Mejor Modelo BETO al Set de Evaluación REST-MEX 2025
Este notebook aplica el modelo de lenguaje BETO previamente entrenado y validado al conjunto de evaluación (test) del reto REST-MEX 2025. Se realiza la carga y preprocesamiento de los datos de test, la generación de predicciones de polaridad para cada instancia y la exportación de los resultados en el formato requerido para su entrega. El flujo asegura la reproducibilidad y facilita la integración de los resultados del modelo en la competencia.

### Cargamos archivo de train/test limpio

In [9]:
import pandas as pd
import numpy as np
import os
# Ruta de lectura
ruta = r"C:\Users\uzgre\Codes\Python\Ciencia de Datos\Proyecto_final\Rest-Mex_2025_DataSet"
archivo = os.path.join(ruta, "Train_Limpio.csv") 

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)


In [10]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208051 entries, 0 to 208050
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Polarity      208051 non-null  float64
 1   Town          208051 non-null  object 
 2   Region        208051 non-null  object 
 3   Type          208051 non-null  object 
 4   Texto_Leido   208051 non-null  object 
 5   Texto_Limpio  208051 non-null  object 
dtypes: float64(1), object(5)
memory usage: 9.5+ MB


In [3]:
Data['Texto_Leido'][10:20]

10    Divertido lugar para pasar el rato Lo pasamos ...
11    BIEN UBICADO Fue mi primera vez en este hotel,...
12    Asegúrate de pasear y explorar Hay un montón d...
13    Increible Increíble, no se puede describir de ...
14    PÉSIMA EXPERIENCIA, NADA RECOMENDABLE. Solo es...
15    Mellow Vibe supera la multitud La playa está l...
16    "La ciudad de los dioses" Ubicado al noreste d...
17    "Nice place" El hotel está limpio pero te sien...
18    Visita obligatoria en San Cristóbal! Hicimos e...
19    Increíble Los cenotes son increíbles, el agua ...
Name: Texto_Leido, dtype: object

In [ ]:
#Revisamos las metricas de un chekpoint (necesario cargar el set de entrenamiento)
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer
from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
from datasets import Dataset

# 1. Preparación del Dataset
Data['Polarity'] = Data['Polarity'].astype(int)
Data = Data[Data['Polarity'].isin([1, 2, 3, 4, 5])]
Data['labels'] = Data['Polarity'] - 1  # Etiquetas 0 a 4

# Convertir a Dataset de Hugging Face
dataset = Dataset.from_pandas(Data[['Texto_Leido', 'labels']])

# 2. Tokenización
tokenizer = BertTokenizerFast.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

def tokenize(batch):
    return tokenizer(batch["Texto_Leido"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

# 3. Separar en evaluación y entrenamiento
dataset = dataset.train_test_split(test_size=0.2, seed=42)
eval_dataset = dataset["test"]

# 4. Cargar modelo desde el checkpoint
#model = BertForSequenceClassification.from_pretrained("./resultados_beto_mas_epocas/checkpoint-41612")
model = BertForSequenceClassification.from_pretrained("./resultados_beto/checkpoint-20806")

# 5. Crear Trainer solo para evaluar
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
)

# 6. Obtener predicciones
predictions = trainer.predict(eval_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

# 7. Calcular métricas
acc = accuracy_score(labels, preds)
f1_macro = f1_score(labels, preds, average="macro")
f1_micro = f1_score(labels, preds, average="micro")
f1_weighted = f1_score(labels, preds, average="weighted")

print(f"Accuracy: {acc:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Weighted: {f1_weighted:.4f}")
print("\nReporte completo:\n")
print(classification_report(labels, preds, digits=4))

Map:   0%|          | 0/208051 [00:00<?, ? examples/s]

C:\Users\uzgre\AppData\Local\Temp\ipykernel_8564\1195260577.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Accuracy: 0.7715
F1 Macro: 0.6953
F1 Micro: 0.7715
F1 Weighted: 0.7780

Reporte completo:

              precision    recall  f1-score   support

           0     0.7195    0.8654    0.7857      1070
           1     0.6489    0.5452    0.5926      1139
           2     0.7182    0.5850    0.6448      3072
           3     0.5257    0.6743    0.5908      9073
           4     0.8975    0.8306    0.8628     27257

    accuracy                         0.7715     41611
   macro avg     0.7020    0.7001    0.6953     41611
weighted avg     0.7918    0.7715    0.7780     41611



### Aplicamos el modelo al Set de prueba/Test

In [ ]:
import torch
print(torch.cuda.is_available())  # True si está bien
print(torch.cuda.get_device_name(0))  # Nombre de tu GPU

In [ ]:
import torch
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Ruta de lectura
ruta = r"C:\Users\uzgre\Codes\Python\Ciencia de Datos\Proyecto_final\Rest-Mex_2025_DataSet"
archivo = os.path.join(ruta, "Test_Limpio.csv") 

# Cargar el archivo CSV de datos de test

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data_test = pd.read_csv(f)

# Cargar modelo y tokenizer
checkpoint_path = r"C:\Users\uzgre\Codes\Python\Ciencia de Datos\Proyecto_final\resultados_Polairty\checkpoint-20806"  # El checkpoint del mejor modelo
# Cargar el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Enviar a dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Tokenizar entradas
inputs = tokenizer(
    list(Data_test['Texto_Leido']),  # Usamos 'Texto_Leido' para la predicción
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors="pt"
)

batch_size = 4  # Reduce el tamaño del batch
inputs = {k: v.to(device) for k, v in inputs.items()}

# Divide en batches más pequeños
preds = []
for i in range(0, len(inputs['input_ids']), batch_size):
    batch_inputs = {k: v[i:i+batch_size] for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**batch_inputs)
        batch_preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        preds.extend(batch_preds)

# Generar archivo de salida en formato .txt 
output_file = "CorpusChristi_ID_Run.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for idx, row in Data_test.iterrows():
        instance_id = row['ID']
        predicted_class = preds[idx] + 1  # De 0–4 a 1–5

        # Línea con formato: rest-mex  ID  predicción  ciudad  categoría
        output_line = f"rest-mex  {instance_id}  {predicted_class}\n"
        f.write(output_line)

print(f"Archivo '{output_file}' generado exitosamente.")
